# GRMP Attack Experiment - Google Colab

This notebook runs the Graph Representation-based Model Poisoning (GRMP) attack experiment on AG News dataset.

**Paper**: Graph Representation-based Model Poisoning on the Heterogeneous Internet of Agents

## Setup Instructions

1. **Enable GPU**: Runtime → Change runtime type → GPU
2. **Upload Files**: Upload all Python files (main.py, client.py, server.py, data_loader.py, models.py, visualization.py) to Colab
3. **Run all cells**: Runtime → Run all
4. **View results**: Check the `results/` folder for outputs and visualizations


## Step 1: Install Dependencies


In [ ]:
# Install required packages
%pip install torch>=2.0.0 transformers>=4.35.0 datasets>=2.0.0 numpy>=1.21.0 scikit-learn>=1.0.0 pandas>=1.3.0 tqdm>=4.62.0 matplotlib>=3.4.0 seaborn>=0.11.0 --quiet

print("✅ Dependencies installed successfully!")


## Step 2: Verify Files and GPU


In [ ]:
# Check if files exist
import os
from pathlib import Path

required_files = ['main.py', 'client.py', 'server.py', 'data_loader.py', 'models.py', 'visualization.py']
missing_files = [f for f in required_files if not os.path.exists(f)]

if missing_files:
    print(f"⚠️  Missing files: {missing_files}")
    print("Please upload these files to Colab using the file uploader.")
else:
    print("✅ All required files found!")
    for f in required_files:
        print(f"  - {f}")

# Check GPU
import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  No GPU detected. Training will be slower.")
    print("   Go to Runtime → Change runtime type → GPU")


## Step 3: Configure Experiment

Choose one of the configurations below:
- **Quick Test**: Faster execution (~10-15 min), reduced rounds and dataset
- **Full Experiment**: Complete experiment (~1-2 hours), full dataset


In [ ]:
# Experiment Configuration
# Modify these parameters as needed

EXPERIMENT_CONFIG = {
    # ========== Experiment Configuration ==========
    'experiment_name': 'colab_grmp_attack',  # Name for result files
    'seed': 42,  # Random seed for reproducibility
    
    # ========== Federated Learning Setup ==========
    'num_clients': 6,  # Total number of federated learning clients
    'num_attackers': 2,  # Number of attacker clients
    'num_rounds': 20,  # Total number of federated learning rounds (reduce to 5 for quick test)
    
    # ========== Training Hyperparameters ==========
    'client_lr': 2e-5,  # Learning rate for local client training
    'server_lr': 0.8,  # Server learning rate for model aggregation
    'batch_size': 16,  # Batch size for local training
    'local_epochs': 5,  # Number of local training epochs per round
    'alpha': 0.01,  # Proximal regularization coefficient
    
    # ========== Data Distribution ==========
    'dirichlet_alpha': 0.5,  # Non-IID distribution parameter
    'test_sample_rate': 1.0,  # Rate of Business samples to test
    'dataset_size_limit': None,  # None = full dataset, or set to int (e.g., 10000) for faster test
    
    # ========== Attack Configuration ==========
    'poison_rate': 1.0,  # Base poisoning rate for attack phase
    'attack_start_round': 10,  # Round when attack phase starts
    
    # ========== Formula 4 Constraint Parameters ==========
    'd_T': 0.5,  # Distance threshold for constraint (4b)
    'gamma': 10.0,  # Upper bound for constraint (4c)
    
    # ========== VGAE Training Parameters ==========
    'dim_reduction_size': 10000,  # Dimensionality for feature reduction (adjust based on GPU memory)
    'vgae_epochs': 20,  # Number of epochs for VGAE training
    'vgae_lr': 0.01,  # Learning rate for VGAE optimizer
    'vgae_lambda': 0.5,  # Weight for preservation loss
    
    # ========== Camouflage Optimization Parameters ==========
    'camouflage_steps': 30,  # Number of optimization steps for camouflage
    'camouflage_lr': 0.1,  # Learning rate for camouflage optimization
    'lambda_proximity': 1.0,  # Weight for constraint (4b) proximity loss
    'lambda_aggregation': 0.5,  # Weight for constraint (4c) aggregation loss
    
    # ========== Graph Construction Parameters ==========
    'graph_threshold': 0.5,  # Threshold for graph adjacency matrix binarization
    
    # ========== Defense Mechanism Parameters ==========
    'defense_threshold': 0.10,  # Base threshold for defense mechanism
    'similarity_alpha': 0.7,  # Weight for pairwise similarities
    
    # ========== Visualization ==========
    'generate_plots': True,  # Whether to generate visualization plots
    'run_both_experiments': False,  # Set to True to run baseline + attack (for Figure 5)
    'run_attack_only': False,  # Set to True to only run attack experiment
}

print("✅ Configuration loaded!")
print(f"\nExperiment: {EXPERIMENT_CONFIG['experiment_name']}")
print(f"Rounds: {EXPERIMENT_CONFIG['num_rounds']}")
print(f"Dataset limit: {EXPERIMENT_CONFIG['dataset_size_limit'] or 'Full dataset'}")


In [ ]:
# Quick Test Configuration (uncomment to use)
# QUICK_TEST_CONFIG = {
#     'experiment_name': 'colab_quick_test',
#     'seed': 42,
#     'num_clients': 6,
#     'num_attackers': 2,
#     'num_rounds': 5,  # Reduced rounds
#     'client_lr': 2e-5,
#     'server_lr': 0.8,
#     'batch_size': 16,
#     'local_epochs': 5,
#     'alpha': 0.01,
#     'dirichlet_alpha': 0.5,
#     'test_sample_rate': 1.0,
#     'dataset_size_limit': 10000,  # Limited dataset
#     'poison_rate': 1.0,
#     'attack_start_round': 3,
#     'd_T': 0.5,
#     'gamma': 10.0,
#     'dim_reduction_size': 5000,
#     'vgae_epochs': 10,
#     'vgae_lr': 0.01,
#     'vgae_lambda': 0.5,
#     'camouflage_steps': 20,
#     'camouflage_lr': 0.1,
#     'lambda_proximity': 1.0,
#     'lambda_aggregation': 0.5,
#     'graph_threshold': 0.5,
#     'defense_threshold': 0.10,
#     'similarity_alpha': 0.7,
#     'generate_plots': True,
#     'run_both_experiments': False,
#     'run_attack_only': False,
# }

# To use quick test: EXPERIMENT_CONFIG = QUICK_TEST_CONFIG


## Step 4: Run Experiment


In [ ]:
# Import and run the experiment
import sys
import warnings
warnings.filterwarnings('ignore')

# Import modules
from main import run_experiment, analyze_results

# Run experiment
print("🚀 Starting GRMP Attack Experiment...")
print("=" * 60)

try:
    results, metrics = run_experiment(EXPERIMENT_CONFIG)
    
    # Analyze results
    analyze_results(metrics)
    
    print("\n✅ Experiment completed successfully!")
except Exception as e:
    print(f"\n❌ Experiment failed: {e}")
    import traceback
    traceback.print_exc()


## Step 5: View Results and Visualizations


In [ ]:
# Display visualization plots
from IPython.display import Image, display
from pathlib import Path

results_dir = Path("results")
experiment_name = EXPERIMENT_CONFIG['experiment_name']

# List of figures to display
figures = [
    ("Figure 3: Global Accuracy and ASR", f"{experiment_name}_figure3.png"),
    ("Figure 4: Cosine Similarity", f"{experiment_name}_figure4.png"),
    ("Figure 5: Local Accuracy (No Attack)", f"{experiment_name}_figure5.png"),
    ("Figure 6: Local Accuracy (With Attack)", f"{experiment_name}_figure6.png"),
]

print("📊 Displaying Visualization Figures:")
print("=" * 60)

for fig_title, fig_name in figures:
    fig_path = results_dir / fig_name
    if fig_path.exists():
        print(f"\n✅ {fig_title}")
        display(Image(str(fig_path)))
    else:
        print(f"\n⚠️  {fig_title} not found")


In [ ]:
# Load and display experiment results summary
import json
from pathlib import Path

results_dir = Path("results")
experiment_name = EXPERIMENT_CONFIG['experiment_name']
results_path = results_dir / f"{experiment_name}_results.json"

if results_path.exists():
    with open(results_path, 'r') as f:
        results_data = json.load(f)
    
    print("📊 Experiment Results Summary:")
    print("=" * 60)
    
    # Display key metrics
    rounds = results_data['progressive_metrics']['rounds']
    clean_acc = results_data['progressive_metrics']['clean_acc']
    attack_asr = results_data['progressive_metrics']['attack_asr']
    
    print(f"\nTotal Rounds: {len(rounds)}")
    print(f"Final Clean Accuracy: {clean_acc[-1]:.4f}")
    print(f"Final Attack Success Rate (ASR): {attack_asr[-1]:.4f}")
    print(f"Peak ASR: {max(attack_asr):.4f}")
    
    # Display per-round summary
    print("\n📈 Per-Round Summary:")
    print("Round | Clean Acc | ASR")
    print("-" * 30)
    for i, (r, acc, asr) in enumerate(zip(rounds, clean_acc, attack_asr)):
        if i % 5 == 0 or i == len(rounds) - 1:  # Show every 5th round and last round
            print(f"{r:5d} | {acc:9.4f} | {asr:.4f}")
    
    # Display local accuracies if available
    if 'local_accuracies' in results_data and results_data['local_accuracies']:
        print("\n📊 Local Accuracies (Last Round):")
        local_accs = results_data['local_accuracies']
        for client_id, accs in sorted(local_accs.items()):
            if accs:
                print(f"  Client {client_id}: {accs[-1]:.4f}")
else:
    print(f"⚠️  Results file not found: {results_path}")


## Step 6: Download Results


In [ ]:
# Create a zip file with all results
import zipfile
from pathlib import Path

results_dir = Path("results")
zip_path = "grmp_experiment_results.zip"

if results_dir.exists():
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file_path in results_dir.rglob('*'):
            if file_path.is_file():
                zipf.write(file_path, file_path.relative_to(results_dir.parent))
    
    print(f"✅ Created zip file: {zip_path}")
    print(f"\n📥 Download the file using the cell below")
else:
    print("⚠️  Results directory not found.")


In [ ]:
# Download results zip file
from google.colab import files

if Path("grmp_experiment_results.zip").exists():
    files.download('grmp_experiment_results.zip')
    print("✅ Download started!")
else:
    print("⚠️  Zip file not found. Run the previous cell first.")
